<a href="https://colab.research.google.com/github/WillDera/cats_vs_dog/blob/master/cats_and_dogs_convNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
TRAIN_DIR = "/content/gdrive/My Drive/Kaggle/cats_dogs_dataset/training_set/training_set"
TEST_DIR = "/content/gdrive/My Drive/Kaggle/cats_dogs_dataset/test_set/test_set"

In [17]:
!pip3 install pillow

In [18]:

# baseline model for the dogs vs cats dataset
import sys
import PIL
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator

In [20]:
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(16, (5, 5), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(500, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(300, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='sigmoid'))
 
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [13]:
# plot diagnostic learning curve
def summarize_diagnostics(history):
  # plot loss
  pyplot.subplot(211)
  pyplot.title("binary_crossentropy lost")
  pyplot.plot(history.history['loss'], color='blue', label='train')
  pyplot.plot(history.history['val_loss'], color='orange', label='test')
 
  # plot accuracy
  pyplot.subplot(212)
  pyplot.title("classification accurcay")
  pyplot.plot(history.history['accuracy'], color='blue', label='train')
  pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 
  # save plot to file
  filename = sys.argv[0].split('/')[-1]
  pyplot.savefig(filename + '_plot.png')
  pyplot.close()

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
  # define model
  model = define_model()
 
  # create data generator
  datagen = ImageDataGenerator(rescale=1.0/255.0)
 
  # prepare iterators
  train_it = datagen.flow_from_directory(TRAIN_DIR, class_mode='binary', batch_size=64, target_size=(28, 28))
  test_it = datagen.flow_from_directory(TEST_DIR, class_mode='binary', batch_size=64, target_size=(28, 28))
 
  # fit model
  history = model.fit_generator(train_it, steps_per_epoch=len(train_it), 
                                validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
  
  # evaluate model
  _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
  print('> %.3f' % (acc * 100.0))
 
  # learning curves
  summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 43 could not be retrieved. It could be because a worker has died.
  UserWarning)
